Author: Jonathan Lim \
Date Written: 12 June 2021

In [1]:
#Basic Imports
import pandas as pd 
import numpy as np
import time
from random import random

#Plot 
import matplotlib.pyplot as plt
%matplotlib inline

#Packages for interpolation 
from scipy.interpolate import interp1d
from scipy import arange, array, exp
import scipy

#Get File directory
import glob

#Import Scikit learn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Import function to create training and test set splits
from sklearn.model_selection import train_test_split

# Import function to automatically create polynomial features (never used)
from sklearn.preprocessing import PolynomialFeatures

# Import Linear Regression and a regularized regression function (never used)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, Lasso, Ridge

#import function to make a machine learning pipeline (never used)
from sklearn.pipeline import make_pipeline

#Display 
pd.options.display.max_rows = 500

## *Script Overview:*

1) Iterate through Excel Sheets 

2) Iterate through Country in Excel Sheet 

3) Interpolate/ Extrapolate Countries with >22 data points (~70% Total Data Points per country) 

4) For years with values, train_test_split was used to pick out 80% data with least model error (reduce impact of anamolies) 

5) Determine which degree polynomial to model data (1 < degree < 3), by choosing lowest RMSE

6) Where negative values for factor is unintuitive, the predicted value is clipped to 0

7) For factor 'WB_FDI_as___of_GNI_', predicted negative value is kept 

8) Export edited Dataset as .csv

In [3]:
start = time.perf_counter()


for excel_directory in lst_excel_directory:
    start_time_2 = time.perf_counter()
    
    #excel name
    excel_name = excel_directory.split('/')[-1].replace(' _.xlsx', '').replace('_.xlsx', '').replace('.xlsx', '').replace('.xls', '').replace(' ', '_')

    #import file
    if excel_directory[-3:] == 'xls':
        excel = pd.read_excel(excel_directory)
    else:
        excel = pd.read_excel(excel_directory, engine='openpyxl')
    
    #make sure only columns required kept
    excel = excel.iloc[:,:5]
        
    #remove rows without country 
    excel = excel[~excel['Country Name'].isna()]
    # factor column name
    col = excel.columns[-1]
    #change nd to 0
    excel[col] = excel[col].replace('ND',np.nan)
    #change string to float
    excel[col] = excel[col].astype('float')
    #for df later to merge with new extrapolated data 
    cols = excel.columns[:-1].tolist() 
    

    #create dictionary to combine all dataframe
    country_df_list = {}
    
    iteration = 1
    
    for country in excel[['Country Name']].drop_duplicates(subset='Country Name').values.tolist():
        #create df
        df = excel[excel['Country Name'] == country[0]]

        #create x and y axis (to plot)
        x_data = df['Year'].values.tolist()
        
        #create df to change (np.nan to 0) #for RMSE and for plotting
        df_change_nan = df.copy()
        df_change_nan[col] = df_change_nan[col].replace(np.nan,0)
        y_data = df_change_nan.loc[:,col].values.tolist() #unnessary unless want to plot

        #create df to exclude datapoints without value
        df_int_ext = df[~df[col].isna()]

        if df_int_ext[[col]].count()[0]<22:
            if iteration ==1:
                country_df_list['combined_df'] = df
                iteration += 1
            else:
                country_df_list['combined_df'] = country_df_list['combined_df'].append(df)
                
        elif df_int_ext[[col]].count()[0]>=22:
            x_data_interpolate = df_int_ext['Year'].values.tolist()
            y_data_interpolate = df_int_ext.loc[:,col].values.tolist()
            
            #lowest rmse score
            lowest_rmse = 100
            polynomial_degree = 1

            #testing which power has the smallest RMSE is best
            for i in range(1,4):
                
                lowest_error = 10000
                random_state_id = 1
                for random_rerun in range(0,10):
                    #split train test
                    X_train, X_test, y_train, y_test = train_test_split(x_data_interpolate, y_data_interpolate, test_size=0.2, random_state = random_rerun)

                    #training model
                    poly_fit = np.poly1d(np.polyfit(X_train,y_train,i))

                    #test model
                    predicted_y = poly_fit(X_test) 
                    error=np.mean((predicted_y-y_test)**2)

                    if error < lowest_error:
                        lowest_error = error
                        random_state_id = iteration
                
                #use these X-train with lowest error
                X_train, X_test, y_train, y_test = train_test_split(x_data_interpolate, y_data_interpolate, test_size=0.2, random_state = random_state_id)
            
                #polynomial fit
                poly_fit = np.poly1d(np.polyfit(X_train,y_train,i))

                other_x = np.array(x_data) #create array for x-axis
                other_y = poly_fit(other_x) #extrapolate/ interpolate y-axis
               
                #Calculated RMSE
                rmse = np.sqrt(mean_squared_error(y_data,other_y))
            
                if rmse < lowest_rmse:
                    lowest_rmse = rmse
                    polynomial_degree = i
            

            #Generate polynomial model
            poly_fit = np.poly1d(np.polyfit(X_train,y_train,polynomial_degree))

            other_x = np.array(x_data) #create array for x-axis
            other_y = poly_fit(other_x) #extrapolate/ interpolate y-axis
            
            if excel_name == 'WB_FDI_as___of_GNI_':
                pass
            else:
                #ground negative values to zero 
                other_y = other_y.clip(min=0) 

            #df containing Year & Factor
            fill_data_df = pd.DataFrame(np.hstack((other_x[:,None],other_y[:,None]))).rename(columns = {0:'Year',1:col})

            #merge based on Year to form new df of data
            new_data_df = df[cols].merge(fill_data_df, how='left', on='Year')
            
            if iteration ==1:
                country_df_list['combined_df'] = new_data_df
                iteration +=1
            else:
                country_df_list['combined_df'] = country_df_list['combined_df'].append(new_data_df)
                 

    #export file 
    country_df_list['combined_df'].to_csv('/Users/jonathanlim/Coding/Mannat/output_data/'+ excel_name + '_edited.csv')
    
    #inform
    print(f'Done for \033[1m \033[91m {excel_name}\033[0m.')
    print(f"Time taken: \033[1m{time.perf_counter()- start_time_2}\033[0m seconds")
    print("="*70)
    
print("*"*70)
print(f'\033[92m\033[1mFull Calculations Done!\033[0m Time taken: \033[1m \033[91m{time.perf_counter()- start}\033[0m seconds')

Done for   WB_Mobile_Phone_Subscriptions.
Time taken: 7.154751798999996 seconds
Done for   WB_weighted_tarrifs_.
Time taken: 3.321757809000019 seconds
Done for   WB_FDI_as___of_GNI_.
Time taken: 6.574082147000013 seconds
Done for   WB___of_taxes_as_part_of_GDP_.
Time taken: 3.340903279000031 seconds
Done for   Societal_Enablement.
Time taken: 4.51494353999999 seconds
Done for   Public_education_expenditure.
Time taken: 1.6352297419999786 seconds
Done for   WB___Conservation_Area_.
Time taken: 1.6805992500000002 seconds
Done for   WB_Number_of_mobile_cellular_subscriptions_per_100.
Time taken: 5.686037747 seconds
Done for   WB_Fixed_broadband_subscriptions_per_100_people_.
Time taken: 1.9649378149999848 seconds
Done for   WB_Incidence_of_Malaria_per_1000_at_risk_.
Time taken: 1.7339531269999497 seconds
Done for   UN_Proportion_of_seats_held_by_women_in_national_parliaments_(__of_total_number_of_seats).
Time taken: 2.6532810930000323 seconds
Done for   WB_Neonatal_mortality_rate_per_1000

## Check

In [54]:
check_df = pd.read_csv('/Users/jonathanlim/Coding/Mannat/output_data/WB_Mobile_Phone_Subscriptions_edited.csv')

#Plot graph for country new data
for country in check_df[['Country Name']].drop_duplicates(subset='Country Name').values.tolist():
    country_df = check_df[check_df['Country Name'] == country[0]]
    
    plt.figure()
    plt.title(f'Trend for {country[0]}')
    plt.plot(country_df['Year'].tolist(),country_df['Mobile Phone Subscriptions %'].tolist())